<a href="https://colab.research.google.com/github/robertozerbini/tfserving_k8s/blob/master/TFServing_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Dependencies

!pip install python-resize-image
from resizeimage import resizeimage
import shutil # save img locally
from __future__ import print_function

import base64
import io
import json

import numpy as np
from PIL import Image
import requests

import tensorflow as tf

import scipy.misc
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

#Resnet-50

In [ ]:
#@title Predict Resnet-50
IMAGE_URL = "https://www.metoffice.gov.uk/binaries/content/gallery/metofficegovuk/hero-images/advice/maps-satellite-images/satellite-image-of-globe.jpg" #@param {type:"string"}
# The server URL specifies the endpoint of your server running the ResNet
# model with the name "resnet" and using the predict interface.
SERVER_URL = 'http://\u003CLoadbalancer IP>:8501/v1/models/resnet:predict' #@param {type:"string"}

#IMAGE_URL = 'https://www.thesprucepets.com/thmb/lIRYTymFwujCVbg4UbxAOyWwQ_s=/5064x3315/filters:fill(auto,1)/canaan-dog--lying-down-110056364-5c44fecac9e77c0001a2bf1c.jpg'

# Download the image
#dl_request = requests.get(IMAGE_URL, stream=True)
#dl_request.raise_for_status()



#IMAGE_URL = 'https://www.thesprucepets.com/thmb/lIRYTymFwujCVbg4UbxAOyWwQ_s=/5064x3315/filters:fill(auto,1)/canaan-dog--lying-down-110056364-5c44fecac9e77c0001a2bf1c.jpg'
file_name = '/tmp/test.jpg'
resized_image = '/tmp/output.jpg'

# Download the image
dl_request = requests.get(IMAGE_URL, stream=True)
dl_request.raise_for_status()
#resize input
with open(file_name,'wb') as f:
    shutil.copyfileobj(dl_request.raw, f) 

with open(file_name, 'r+b') as f:
    with Image.open(f) as image:
        cover = resizeimage.resize_cover(image, [250, 250])
        cover.save(resized_image, image.format)
        
# Compose a JOSN Predict request (send the image tensor).
#jpeg_rgb = Image.open(io.BytesIO(dl_request.content))
jpeg_rgb = Image.open(resized_image)
# Normalize and batchify the image
jpeg_rgb = np.expand_dims(np.array(jpeg_rgb) / 255.0, 0).tolist()
predict_request = json.dumps({'instances': jpeg_rgb})

response = requests.post(SERVER_URL, data=predict_request)
response.raise_for_status()
#total_time += response.elapsed.total_seconds()
prediction = response.json()['predictions'][0]

tf.keras.applications.resnet50.decode_predictions(
    np.array(prediction)[:1000].reshape(1,1000), top=5
)


#Test

In [ ]:
file = open('request.json', 'w')
file.write(predict_request)
file.close()

In [ ]:
!curl -d "@request.json" -X POST SERVER_URL

In [ ]:
load_image_into_numpy_array(resized_image)